In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
#set dtype_map to reduce memory usage
dtype_map = {
    # keep continuous as Float32 (allows NaN)
    "passenger_count": "Float32",
    "trip_distance":   "Float32",
    "extra":           "Float32",
    "total_amount":    "Float32",
    # IDs as nullable integers (capital I)
    "RatecodeID":   "Int16",
    "PULocationID": "Int16",
    "DOLocationID": "Int16",
    "payment_type": "Int8",
}

In [3]:
REQUIRED_COLS = [
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
        "RatecodeID",
        "PULocationID",
        "DOLocationID",
        "payment_type",
        "extra",
        "total_amount",
]

In [6]:
df = pd.read_csv(
    "../data/nytaxi2022.csv",
    usecols=REQUIRED_COLS,
    dtype=dtype_map,
    na_values=["\\N", "", "NA"], # treat these as missing
    keep_default_na=True,    
    low_memory=False
)

In [7]:
dt_fmt = '%m/%d/%Y %I:%M:%S %p'
df["tpep_pickup_datetime"]  = pd.to_datetime(df["tpep_pickup_datetime"], format=dt_fmt, errors="coerce")
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], format=dt_fmt, errors="coerce")


In [8]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,extra,total_amount
0,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1,142,236,1,3.0,21.950001
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1,236,42,1,0.5,13.3
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1,166,166,1,0.5,10.56
3,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1,114,68,2,0.5,11.8
4,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1,68,163,1,0.5,30.299999


In [9]:
print(df.dtypes)
print(type(df["tpep_pickup_datetime"].iloc[0]))

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 Float32
trip_distance                   Float32
RatecodeID                        Int16
PULocationID                      Int16
DOLocationID                      Int16
payment_type                       Int8
extra                           Float32
total_amount                    Float32
dtype: object
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [10]:
#count rows before cleaning
print("Number of rows before cleaning: ", len(df))

Number of rows before cleaning:  39656098


In [11]:
#drop rows with missing values
df = df.dropna()

In [12]:
#count rows after dropping missing values
print("Number of rows after dropping missing values: ", len(df))

Number of rows after dropping missing values:  38287795


In [ ]:
#basic sanity check on passenger_count, trip_distance, total_amount. 
#remove passenger_count <= 0, trip_distance <= 0, total_amount <= 0, extra<=0
df = df[(df["passenger_count"] > 0) & (df["trip_distance"] > 0) & (df["total_amount"] > 0) & (df['extra'] > 0)]

In [ ]:
#number of rows after sanity check
print("Number of rows after removing negative values: ", len(df))

Number of rows after sanity check:  36818628


In [15]:
#add duration column in minutes
df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60.0

In [16]:
#move duration_min column to be after tpep_dropoff_datetime
cols = list(df.columns)

# find index of 'tpep_dropoff_datetime'
idx = cols.index("tpep_dropoff_datetime")

# move duration_min right after tpep_dropoff_datetime
cols.insert(idx+1, cols.pop(cols.index("duration_min")))

df = df[cols]

In [17]:
#drop rows with duration_min <= 0
df = df[df["duration_min"] > 0]

In [18]:
#count rows after dropping impossible durations
print("Number of rows after dropping impossible durations: ", len(df))


Number of rows after dropping impossible durations:  36816761


In [19]:
#given nyc's speed limit of 25mph to 65mph i.e. 40km/h to 105km/h, remove trips impossible trips 
df["duration_hour"] = df["duration_min"] / 60.0
df["speed_kmh"] = df["trip_distance"] / df["duration_hour"] #lets just assume trip distance is in km


In [20]:
cols = list(df.columns)

# find index of 'duration_min'
idx = cols.index("duration_min")

# move duration_hour right after duration_min
cols.insert(idx+1, cols.pop(cols.index("duration_hour")))

# because we just moved a column, recompute index of duration_min
idx = cols.index("duration_min")

# move speed_kmh right after duration_min (which will then be after duration_hour)
cols.insert(idx+2, cols.pop(cols.index("speed_kmh")))

df = df[cols]


In [21]:
#drop nonsense rows, nyc speed limit is 40-105kmh
df = df[(df["speed_kmh"] >= 1) & (df["speed_kmh"] <= 120)]

In [22]:
# count rows after dropping impossible speeds
print("Number of rows after dropping impossible speeds: ", len(df))

Number of rows after dropping impossible speeds:  36724241


In [ ]:
#drop rows with total amount <3, which is the minimum fare in nyc
df = df[df["total_amount"] >= 3]

In [27]:
# count rows after dropping total_amount < 3
print("Number of rows after dropping total_amount < 3: ", len(df))


Number of rows after dropping total_amount < 3:  36720650


In [31]:
#drop rows with duration <= 1
df = df[df["duration_min"] > 1]

In [32]:
#count rows after dropping duraion  <= 1
print("Number of rows after dropping duration <= 1 minute: ", len(df))

Number of rows after dropping duration <= 1 minute:  36604981


In [33]:
#calculate fare_per_km
df["fare_per_km"] = df["total_amount"] / df["trip_distance"]

#calculate fare_per_min
df["fare_per_min"] = df["total_amount"] / df["duration_min"]

/var/folders/t2/8dnrmst93z7fjnvxwhl1yzj40000gn/T/ipykernel_9842/2096913589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fare_per_km"] = df["total_amount"] / df["trip_distance"]
/var/folders/t2/8dnrmst93z7fjnvxwhl1yzj40000gn/T/ipykernel_9842/2096913589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fare_per_min"] = df["total_amount"] / df["duration_min"]


In [39]:
#remove ratecodeid that is not either 1,2,3,4,5,6,99
df = df[df["RatecodeID"].isin([1,2,3,4,5,6,99])]

In [40]:
#count rows after dropping invalid ratecodeid
print("Number of rows after dropping invalid RatecodeID: ", len(df))

Number of rows after dropping invalid RatecodeID:  36348804


In [41]:
# bounds per RatecodeID: (km_low, km_high, min_low, min_high)
bounds = {
    1: (0.5, 20, 0.10, 10),
    2: (0.5, 10, 0.03, 10),
    3: (0.5, 12, 0.05, 12),
    4: (0.5, 20, 0.05, 12),
    5: (0.05, 30, 0.02, 20),
    6: (0.05, 20, 0.02, 12),
    99: (0.05, 30, 0.05, 20),
}

# default fallback if unseen code appears
default_bounds = (0.05, 30, 0.05, 20)

# vectorized mask builder
def make_mask(s_code, km, per_min):
    km_lo, km_hi, m_lo, m_hi = zip(*[
        bounds.get(code, default_bounds) for code in s_code.values
    ])
    km_lo = np.array(km_lo); km_hi = np.array(km_hi)
    m_lo  = np.array(m_lo);  m_hi  = np.array(m_hi)
    return (km >= km_lo) & (km <= km_hi) & (per_min >= m_lo) & (per_min <= m_hi)

mask = make_mask(df["RatecodeID"], df["fare_per_km"].to_numpy(), df["fare_per_min"].to_numpy())

df = df[mask]

In [42]:
#count rows after dropping invalid rates
print("Number of rows after dropping invalid rates: ", len(df))

Number of rows after dropping invalid rates:  35437231


In [38]:
df.describe()

,tpep_pickup_datetime,tpep_dropoff_datetime,duration_min,duration_hour,speed_kmh,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,extra,total_amount,fare_per_km,fare_per_min
count,36348804,36348804,3.634880e+07,3.634880e+07,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0,36348804.0
mean,2022-07-04 18:22:49.189546752,2022-07-04 18:38:43.504368896,1.590525e+01,2.650875e-01,12.068337,1.430774,3.5629,1.343532,164.986494,162.764035,1.210262,1.023973,21.485229,9.041125,1.554202
min,2001-01-01 00:03:14,2001-01-01 00:34:17,1.016667e+00,1.694444e-02,1.000129,1.0,0.1,1.0,1.0,1.0,1.0,0.0,3.0,0.075793,0.050082
25%,2022-04-07 15:02:48,2022-04-07 15:22:13.750000128,7.616667e+00,1.269444e-01,7.837638,1.0,1.16,1.0,132.0,113.0,1.0,0.0,12.3,5.959596,1.141791
50%,2022-06-30 17:35:08,2022-06-30 17:54:10,1.233333e+01,2.055556e-01,10.336633,1.0,1.9,1.0,162.0,162.0,1.0,0.5,15.96,8.194445,1.399052
75%,2022-10-04 12:27:58.500000,2022-10-04 12:46:07,1.983333e+01,3.305556e-01,14.160363,1.0,3.57,1.0,234.0,234.0,1.0,2.5,22.77,11.070968,1.78125
max,2023-01-01 00:31:23,2023-01-01 00:56:58,3.001450e+03,5.002417e+01,120.0,9.0,380.130005,99.0,265.0,265.0,5.0,33.5,1783.849976,30.0,19.993356
std,NaN,NaN,1.388525e+01,2.314208e-01,6.593852,0.953659,4.508173,5.287256,64.834422,70.139865,0.427119,1.253535,16.725527,4.262414,0.646493


tpep_pickup_datetime: The date and time when the passenger was picked up.

tpep_dropoff_datetime: The date and time when the passenger was dropped off.

passenger_count: The number of passengers in the taxi.

trip_distance: The total distance of the trip in miles or kilometers.

RatecodeID: The rate code assigned to the trip, representing fare types.

PULocationID: The unique identifier for the pickup location (zone or area).

DOLocationID: The unique identifier for the drop-off location (zone or area).

payment_type: The method of payment used by the passenger (e.g., cash, card).

extra: Additional charges applied during the trip (e.g., night surcharge).

total_amount: The total fare amount, including all charges and surcharges.


In [24]:
#show summary of eah column
print(df.describe())

                tpep_pickup_datetime          tpep_dropoff_datetime  \
count                       36724241                       36724241   
mean   2022-07-04 22:54:34.689054208  2022-07-04 23:10:21.876974336   
min              2001-01-01 00:03:14            2001-01-01 00:34:17   
25%              2022-04-07 16:59:04            2022-04-07 17:18:17   
50%              2022-06-30 20:59:43            2022-06-30 21:14:56   
75%              2022-10-04 16:52:31            2022-10-04 17:10:25   
max              2023-01-01 00:31:23            2023-01-01 00:56:58   
std                              NaN                            NaN   

       duration_min  duration_hour   speed_kmh  passenger_count  \
count  3.672424e+07   3.672424e+07  36724241.0       36724241.0   
mean   1.578647e+01   2.631078e-01   12.037744         1.430675   
min    1.666667e-02   2.777778e-04         1.0              1.0   
25%    7.516667e+00   1.252778e-01    7.792208              1.0   
50%    1.225000e+01   2.0

In [43]:
#output cleaned data to csv
df.to_csv("../data/nytaxi2022_cleaned.csv", index=False)

In [44]:
#split into train and test set, 70% train, 30% test
import hashlib
import numpy as np
import pandas as pd

TRAIN_RATIO = 0.7
OUTPUT_DIR = "../data/"   # adjust path as needed

#split function
def assign_split(keys, train_ratio=TRAIN_RATIO):
    """
    Deterministic hash-based split into train/test.
    keys: pd.Series of string identifiers for each row
    """
    hash_vals = keys.apply(
        lambda s: int(hashlib.blake2b(s.encode(), digest_size=8).hexdigest(), 16) % 10000
    )
    return np.where(hash_vals < int(train_ratio * 10000), "train", "test")
#Build keys for hashing, use only stable and unique columns (no NaNs, already cleaned)
keys = (
    df["tpep_pickup_datetime"].astype("int64").astype(str) + "|" +
    df["tpep_dropoff_datetime"].astype("int64").astype(str) + "|" +
    df["PULocationID"].astype(str) + "|" +
    df["DOLocationID"].astype(str) + "|" +
    df["trip_distance"].round(4).astype(str) + "|" +
    df["total_amount"].round(2).astype(str)
)

# Apply split
df["split"] = assign_split(keys)

train_df = df[df["split"] == "train"].copy()
test_df  = df[df["split"] == "test"].copy()

print(f"Train size: {len(train_df):,} rows")
print(f"Test size:  {len(test_df):,} rows")
print(f"Train ratio: {len(train_df) / len(df):.3f}")

# Export
train_df.to_parquet(f"{OUTPUT_DIR}/taxi_train.parquet", index=False)
test_df.to_parquet(f"{OUTPUT_DIR}/taxi_test.parquet", index=False)

# Optional: also CSV (larger files, slower IO)
train_df.to_csv(f"{OUTPUT_DIR}/taxi_train.csv", index=False)
test_df.to_csv(f"{OUTPUT_DIR}/taxi_test.csv", index=False)

print("Export complete! Files written to:", OUTPUT_DIR)


Train size: 24,807,483 rows
Test size:  10,629,748 rows
Train ratio: 0.700
Export complete! Files written to: ../data/
